In [1]:
!pip install numpyencoder

In [2]:
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit
import random
import math
import json
import os
from pathlib import Path
import time
import glob
#LINK_PROJECT = Path(os.path.abspath(__file__)).parent.parent
from numpyencoder import NumpyEncoder

In [3]:
def IP(data, time_limit):
    # Opening JSON file
    N = data['N']; m = data['m']; M = data['M']; d = data['d']; s = data['s']; e = data['e']
    d = dict(zip([*range(1, N + 1)], d))
    s = dict(zip([*range(1, N + 1)], s))
    e = dict(zip([*range(1, N + 1)], e))

    solution = {}
    solver = pywraplp.Solver.CreateSolver('SAT')
    infinity = solver.infinity()

    last_pos_day = max(e.values()) + 1
    #print(max(e))

    x = dict() # row as field, column as day
    for field in range(1, N + 1):
        x[field] = dict()
        for day in range(last_pos_day):
            x[field][day] = solver.IntVar(0, 1, f'x[{field}][{day}]')

    productivity = dict()
    day_to_harvest = dict()
    for day in range(last_pos_day):
        productivity[day] = solver.IntVar(0, infinity, f'productivity_day_{day}')
        day_to_harvest[day] = solver.IntVar(0, 1, f'harvest_in_day_{day}')

    min_productivity_per_day = solver.IntVar(0, infinity, f'min_productivity_per_day')
    max_productivity_per_day = solver.IntVar(0, infinity, f'max_productivity_per_day')

    for field in range(1, N + 1):
        constraint = solver.RowConstraint(1, 1, f'day to harvest field {field}')
        for day in range(last_pos_day):
            constraint.SetCoefficient(x[field][day], 1)

    # for field in range(1, N + 1):
    #     print(d[field])

    # 1 <= (1 - day_to_harvest)N + x1 + x2 + ... + xN <= N
    for day in range(last_pos_day):
        constraint = solver.RowConstraint(1 - N, 0, f'assure to harvest only when having admission in day {day}')
        constraint.SetCoefficient(day_to_harvest[day], -N)
        for field in range(1, N + 1):
            constraint.SetCoefficient(x[field][day], 1)

    for day in range(last_pos_day):
        # m <= (1 - day_to_harvest)m + x1d1 + x2d2 + ... + xNdN <= M
        constraint = solver.RowConstraint(0, M - m, f'productivity not surpass M and at least m in day {day}')
        constraint.SetCoefficient(day_to_harvest[day], -m)
        for field in range(1, N + 1):
            constraint.SetCoefficient(x[field][day], d[field])

    for day in range(last_pos_day):
        constraint = solver.RowConstraint(0, 0, f'productivity in day {day}')
        constraint.SetCoefficient(productivity[day], -1)
        for field in range(1, N + 1):
            constraint.SetCoefficient(x[field][day], d[field])

    for day in range(last_pos_day):
        # min_productivity_per_day <= M*(1 - day_to_harvest) + productivity
        constraint = solver.RowConstraint(-M, infinity, f'min productivity must not surpass productivity of field {field}')
        constraint.SetCoefficient(day_to_harvest[day], -M)
        constraint.SetCoefficient(min_productivity_per_day, -1)
        constraint.SetCoefficient(productivity[day], 1)

        constraint = solver.RowConstraint(0, infinity, f'max productivity must not below productivity of field {field}')
        constraint.SetCoefficient(productivity[day], -1)
        constraint.SetCoefficient(max_productivity_per_day, 1)

    for field in range(1, N + 1):
        constraint = solver.RowConstraint(s[field], e[field], f'day to harvest must be valid in field {field}')
        for day in range(last_pos_day):
            constraint.SetCoefficient(x[field][day], day)



    objective = solver.Objective()
    objective.SetCoefficient(max_productivity_per_day, 1)
    objective.SetCoefficient(min_productivity_per_day, -1)
    objective.SetMinimization()
    
    if time_limit:
        solver.set_time_limit(time_limit)
    start_time = time.time()
    status = solver.Solve()
    end_time = time.time()
    solution["time"] = end_time - start_time

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        solution["obj"] = objective.Value()
        solution["field"] = []
        for field in range(1, N + 1):
            for day in range(last_pos_day):
                if x[field][day].solution_value() == 1:
                    solution["field"].append(day)
    else:
        #print('No solution!')
        solution["obj"] = "No Solution"
        
    return solution


def import_data(input_file):
    with open(input_file) as f:
        data = json.load(f)
        return data
    
def export(output_file, solution):
    if len(solution) == 3:
        with open(output_file, "w+") as f:
            json.dump({"Time": solution["time"], "Result": solution["obj"],
                      "Solution": solution["field"]}, f, cls=NumpyEncoder)
    else:
        with open(output_file, "w+") as f:
            json.dump({"Time": solution["time"], "Result": solution["obj"]}, f, cls=NumpyEncoder)
    
def process(input_file, output_file, time_limit):
    data = import_data(input_file)
    solution = IP(data, time_limit)
    export(output_file, solution)

In [ ]:
for path in glob.glob("..\\data\\data_v2\\*Small*\\**.json"):
    datatype, name = path.split("\\")[-2:]
    print("Type: ", datatype, " | Name: ", name)
    input_file = f"../data/data_v2/{datatype}/{name}"
    output_file = f"../results/data_v2/{datatype}/integer_programming/result_{name}"
    time_limit = 600000
    process(input_file, output_file, time_limit)

Type:  Type1Small  | Name:  sample_10_10_15.json
Type:  Type1Small  | Name:  sample_10_20_30.json
Type:  Type1Small  | Name:  sample_10_30_50.json
Type:  Type1Small  | Name:  sample_10_40_75.json
Type:  Type1Small  | Name:  sample_10_50_100.json
Type:  Type1Small  | Name:  sample_20_10_15.json
Type:  Type1Small  | Name:  sample_20_20_30.json
Type:  Type1Small  | Name:  sample_20_30_50.json
Type:  Type1Small  | Name:  sample_20_40_75.json
Type:  Type1Small  | Name:  sample_20_50_100.json
Type:  Type1Small  | Name:  sample_30_10_15.json
Type:  Type1Small  | Name:  sample_30_20_30.json
Type:  Type1Small  | Name:  sample_30_30_50.json
Type:  Type1Small  | Name:  sample_30_40_75.json
Type:  Type1Small  | Name:  sample_30_50_100.json
Type:  Type1Small  | Name:  sample_40_10_15.json
Type:  Type1Small  | Name:  sample_40_20_30.json
Type:  Type1Small  | Name:  sample_40_30_50.json
Type:  Type1Small  | Name:  sample_40_40_75.json
Type:  Type1Small  | Name:  sample_40_50_100.json
Type:  Type1Smal